# Batch Scoring Notebook
In this notebook, we step through batch scoring against a deployed machine learning model. There are multiple uses for batch scoring but our focus in this scenario is batch scoring so that Watson Openscale can collect enough data points to be able to provide adequate metrics on fairness and accuracy of the trained model.

In [ ]:
import pandas as pd

In [ ]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient

## WML Deployed Model
In this section, we setup the configuration to access Watson Machine Learning deployed model.

In the next cell, please provide the Watson Machine Learning credentials which you can get by navigating to your IBM Cloud account, finding your Watson Machine Learning service, and clicking on "Service Credentials".
```
wml_credentials = {
    "apikey": "########",
    "instance_id": "#########",
    "url": "##########"
}
```

In [ ]:
# Fill out apikey, instance_id, and url values from your WML service credentials
wml_credentials = {
    "apikey": "#########",
    "instance_id": "##########",
    "url": "#########"
}

In [ ]:
# The code was removed by Watson Studio for sharing.

In [ ]:
client = WatsonMachineLearningAPIClient( wml_credentials )

In [ ]:
client.deployments.list()

#### Identify the right deployment id from the above list corresponding to the deployment to which you want to send the scoring request. 
#### Use the same deployment id to get deployment details in the next cell

In [ ]:
deployment_id = '##################'
deployment_details = client.deployments.get_details(deployment_id)

In [ ]:
# Extract the scoring end point for the churn prediction model
churn_scoring_url = deployment_details['entity']['scoring_url']
print(churn_scoring_url)

## Scoring Data
In this section, we download the scoring data which we will send to the deployed customer churn prediction machine learning model.

The data is stored as a csv file on github so it can be loaded directly into a pandas dataframe using read_csv functionality.


In [ ]:
csvurl = "https://raw.githubusercontent.com/joe4k/customerchurnopenscale/master/data/CustomerChurnScoringData.csv"
churn_data_daily = pd.read_csv(csvurl,header=0)


In [ ]:
churn_data_daily.shape

In [ ]:
# drop ID column
churnDF = churn_data_daily.drop(['ID'],axis=1)

In [ ]:
churnDF.head()

In [ ]:
deployment_details = client.deployments.get_details(deployment_id)
churn_scoring_url = deployment_details['entity']['scoring_url']


In [ ]:
flds = churnDF.columns.tolist()
print(flds)

In [ ]:
# Use only the first 30 records. In practice, you want to monitor all scoring results but for this hands-on lab, 30 records is sufficient to emulate fairness monitor in Openscale.
sample = churnDF[0:30]

In [ ]:
# Print top records
sample.head()

In [ ]:
# Print size/shape of the sample
sample.shape

In [ ]:
values = sample.values.tolist()

In [ ]:
flds = sample.columns.tolist()
values = sample.values.tolist()
payload = {
    "fields": flds,
    "values": values
}

In [ ]:
result = client.deployments.score(churn_scoring_url, payload)
print(result)

## Conclusion
You have completed running batch scoring task for monitoring a churn prediction machine learning model.